In [23]:
import pandas as pd
from sklearn.cluster import MeanShift
import numpy as np
from numpy.linalg import norm

In [24]:
dataa = pd.read_csv('checkins.dat', sep='|', skipinitialspace = True, header=0)
dataa.head()


,id,user_id,venue_id,latitude,longitude,created_at
0,---------+---------+----------+---------------...,NaN,NaN,NaN,NaN,NaN
1,984301,2041916.0,5222.0,NaN,NaN,2012-04-21 17:39:01
2,984222,15824.0,5222.0,38.895112,-77.036366,2012-04-21 17:43:47
3,984315,1764391.0,5222.0,NaN,NaN,2012-04-21 17:37:18
4,984234,44652.0,5222.0,33.800745,-84.410520,2012-04-21 17:43:43


In [25]:
#deleting unnecessary spaces in column names
dataa.rename(columns=lambda x: x.strip(), inplace=True)

dataa.dropna(inplace=True)

#taking only coordinates for future banner placement analysis
dat_coord = dataa[['latitude','longitude']]


In [26]:
#creating two sets of data for long and quick training
X_full=dat_coord.values

X_cut=X_full[:100000,:]


In [27]:
%%time
clustering = MeanShift(bandwidth=0.1).fit(X_cut)

Wall time: 2min 38s


In [28]:
'''
%%time
clustering = MeanShift(bandwidth=0.1).fit(X_full)
'''

'\n%%time\nclustering = MeanShift(bandwidth=0.1).fit(X_full)\n'

In [29]:
'''we are taking clusters with more than 15 elements for calculating distance to the office'''

#labels of each point
labels = clustering.labels_

#matrix of cluster centers coordinates
cluster_centers = clustering.cluster_centers_

#calculating the number of elements in each cluster
labels_unique, counts = np.unique(labels, return_counts=True)
n_clusters_ = len(labels_unique)
print ('the number of clusters: ', n_clusters_)
print (cluster_centers.shape)

#taking clusters with the size >15
dict_count=dict(zip(labels_unique, counts))
labels_for_over_15=[key for key, value in dict_count.items() if value>15]

#taking centers of those classes
centers_for_over_15=cluster_centers[labels_for_over_15]
print (centers_for_over_15)

the number of clusters:  3231
(3231, 2)
[[  40.7177164   -73.99183542]
 [  33.44943805 -112.00213969]
 [  33.44638027 -111.90188756]
 ...
 [  41.61853175  -88.44556818]
 [  39.2494686   -77.1821271 ]
 [  38.65877915  -76.8856871 ]]


In [30]:
#coordinates of offices
offices = np.matrix([[33.751277, -118.188740],[25.867736, -80.324116], [51.503016, -0.075479],[52.378894, 4.885084],[39.366487, 117.036146], [-33.868457, 151.205134]])
print ('offices: \n', offices)
print()

#calculating distances from the centers of clusters to every office
distances=np.zeros((centers_for_over_15.shape[0],offices.shape[0]))

for i,j in enumerate(centers_for_over_15):
    for k,l in enumerate(offices):
        distances[i][k]=norm(j-l)
        
print (distances)
print ()

index_answer=np.unravel_index(np.argmin(distances, axis=None), distances.shape)
print('the closest claster center for placing a banner')
answer=centers_for_over_15[index_answer[0]]
print (answer)

offices: 
 [[ 3.37512770e+01 -1.18188740e+02]
 [ 2.58677360e+01 -8.03241160e+01]
 [ 5.15030160e+01 -7.54790000e-02]
 [ 5.23788940e+01  4.88508400e+00]
 [ 3.93664870e+01  1.17036146e+02]
 [-3.38684570e+01  1.51205134e+02]]

[[ 44.74257092  16.14371999  74.69906582  79.73425538 191.0327603
  237.22725876]
 [  6.19395917  32.5726786  113.37331719 118.41008135 229.11470442
  271.67953165]
 [  6.29424146  32.47447468 113.2748328  118.31160891 229.01456477
  271.58164907]
 ...
 [ 30.76605219  17.72133043  88.92117683  93.94890117 205.49405482
  251.2583223 ]
 [ 41.37357142  13.74564883  78.07422497  83.1108233  194.21830835
  239.80611351]
 [ 41.5935783   13.24513413  77.87671339  82.91381403 193.92312447
  239.34415107]]

the closest claster center for placing a banner
[-33.86063043 151.20477593]


In [31]:
ind = np.unravel_index(np.argsort(distances, axis=None), distances.shape)
print (ind)
#we get the coordinates at 0 index, a label of office at 1 index 

#20 closest locations for placing banners
print ('20 coordinates')
for i in ind[0][:20]:
    print (centers_for_over_15[i])

(array([417, 369, 416, ..., 172, 400, 418], dtype=int64), array([5, 3, 1, ..., 5, 5, 5], dtype=int64))
20 coordinates
[-33.86063043 151.20477593]
[52.37296399  4.89231722]
[ 25.84567226 -80.3188906 ]
[51.50299126 -0.12553729]
[  33.80987796 -118.14892381]
[ 25.78581242 -80.21793804]
[ 25.70534972 -80.28342874]
[ 26.01009825 -80.19999059]
[  33.88832534 -118.04892817]
[  33.87298601 -118.36209115]
[  33.97257482 -118.16837067]
[ 26.13884379 -80.33434684]
[  33.98393587 -118.00740497]
[ 26.12086266 -80.15890668]
[  33.81730643 -117.89124917]
[  34.06039755 -118.24870903]
[  33.67430266 -117.85878927]
[ 26.20058464 -80.25071613]
[  34.03548695 -118.43899772]
[  34.13146015 -118.11801181]
